In [3]:
!pip install psycopg2-binary sqlalchemy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 41.1 MB/s eta 0:00:00


In [4]:
import pandas as pd
import os
from sqlalchemy import create_engine

# Load database credentials from environment variables (with default fallbacks)
db_user = os.getenv("PG_USER", "postgres")
db_pass = os.getenv("PG_PASSWORD", "isba_4715")
db_host = os.getenv("PG_HOST", "isba-dev-02.ctwgc6gcobls.us-east-2.rds.amazonaws.com")
db_port = os.getenv("PG_PORT", "5432")
db_name = os.getenv("PG_DB", "sql_project")

# Create PostgreSQL connection using SQLAlchemy
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}')

# Display all rows in results
pd.set_option('display.max_rows', None)

Descriptive Analytics
Business Question: Which networks have the most high-rated shows, and what is their average rating?

In [5]:
sql_query = '''
SELECT
    "Network",
    COUNT(*) AS high_rating_show_count,
    ROUND(AVG("Rating")::numeric, 2) AS avg_high_rating
FROM public."tvmaze_shows"
WHERE "Rating" > 8.5
GROUP BY "Network"
ORDER BY avg_high_rating DESC
'''


descriptive_df = pd.read_sql(sql_query, engine)
pd.set_option('display.max_rows', None)
descriptive_df

,Network,high_rating_show_count,avg_high_rating
0,Disney XD,1,8.90
1,HBO,1,8.90
2,ITV1,1,8.80
3,BBC One,2,8.75
4,Syndication,1,8.70
5,Syfy,1,8.60
6,FOX,1,8.60


Insight:

Disney XD and HBO top the chart with an average high rating of 8.90 each.

BBC One stands out with 2 high-rated shows, indicating not just high quality but also greater consistency.

Recommendation:

Focus partnership, licensing, or competitive benchmarking around BBC One, HBO, and Disney XD.

HBO and Disney XD may indicate opportunities for high-value original programming insights.

Prediction:

Networks like BBC One are likely to continue producing consistently high-rated content.

HBO and Disney XD may maintain their prestige status with selectively excellent releases, but their impact is more dependent on individual titles.

Diagnostic Business Question: Which networks with high-rated shows tend to premiere those shows more recently, and how does premiere timing relate to the volume of quality content?



In [6]:
sql_query = '''
WITH high_rated_shows AS (
    SELECT *
    FROM tvmaze_shows
    WHERE "Rating" > 8.5
)
SELECT
    "Network",
    ROUND(AVG(EXTRACT(YEAR FROM "Premiered"::timestamp)), 0) AS avg_premiere_year,
    COUNT(*) AS high_rating_show_count
FROM high_rated_shows
GROUP BY "Network"
ORDER BY avg_premiere_year DESC;
'''


diagnostic_df = pd.read_sql(sql_query, engine)
pd.set_option('display.max_rows', None)
diagnostic_df

,Network,avg_premiere_year,high_rating_show_count
0,Disney XD,2012.0,1
1,BBC One,2011.0,2
2,ITV1,2010.0,1
3,HBO,2001.0,1
4,Syfy,1999.0,1
5,FOX,1993.0,1
6,Syndication,1987.0,1


Insight:

Networks like Disney XD, BBC One, and ITV1 are producing high-rated shows that premiered more recently (avg. years: 2012, 2011, 2010), while FOX, Syndication, and HBO have high-rated shows that are comparatively older (avg. years: 1993, 1987, 2001). This implies newer content correlates with high ratings more often.

Recommendation:

Invest in newer releases — prioritize producing and releasing original shows within the past 10 years. Networks with more recent content seem to be achieving higher ratings, suggesting evolving viewer preferences favor current themes, tech, or formats.

Prediction:

Networks that continue releasing fresh high-quality shows (i.e., premiered post-2010) are likely to gain market share and outperform those relying on older libraries. Expect a widening gap in average ratings if older-premiering networks don’t refresh their catalogs.
